In [1]:
import numpy as np
import pandas as pd
from math import sqrt

In [2]:
movei_df=pd.read_csv("movies.csv")
ratings_df=pd.read_csv("ratings.csv")


In [3]:
movei_df['year']=movei_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movei_df['year']=movei_df.year.str.extract('(\d\d\d\d)',expand=False)
movei_df['title']=movei_df.title.str.replace('(\(\d\d\d\d\))','',regex=True)
movei_df['title']=movei_df.title.apply(lambda x : x.strip())
movei_df['genres']=movei_df.genres.str.split('|')
movei_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [4]:
movei_df.drop('genres',axis=1,inplace=True)


In [5]:
movei_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
ratings_df.drop('timestamp',axis=1,inplace=True)

In [8]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [9]:
userInput=[
    {'title':'Breakfast Club, The','rating':5},
    {'title': 'Toy Story','rating':3.5},
    {'title': 'Jumanji' , 'rating':2},
    {'title' : 'Pulp Fiction', 'rating':5},
    {'title':'Akira', 'rating':4.5}
    ]
inputmovies = pd.DataFrame(userInput)
inputmovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [10]:
inputId=movei_df[movei_df['title'].isin(inputmovies['title'].tolist())]
inputmovies=pd.merge(inputId,inputmovies)
inputmovies=inputmovies.drop('year',axis=1)
inputmovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [11]:
usersubset=ratings_df[ratings_df['movieId'].isin(inputmovies['movieId'].tolist())]
usersubset.head()

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


In [12]:
usersubsetgroup=usersubset.groupby(['userId'])


In [13]:
usersubsetgroup.get_group(610)

,userId,movieId,rating
99534,610,1,5.0
99552,610,296,5.0
99636,610,1274,5.0
99664,610,1968,4.0


In [14]:
usersubsetgroup = sorted(usersubsetgroup, key=lambda x : len(x[1]),reverse=True)

In [15]:
usersubsetgroup[0:3]

[(91,
         userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0),
 (177,
         userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5),
 (219,
         userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31554     219      296     4.0
  31628     219     1274     2.5
  31680     219     1968     3.0)]

In [16]:
usersubsetgroup=usersubsetgroup[0:100]

In [17]:
pearsonCorrelationDict = {}
for name, group in usersubsetgroup:
    group=group.sort_values(by='movieId')
    inputmovies = inputmovies.sort_values(by='movieId')
    nRatings = len(group)
    temp_df = inputmovies[inputmovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()

    sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2) /float(nRatings)
    
    sxy = sum(i * j for i, j in zip(tempRatingList, tempGroupList)) -sum(tempRatingList) * sum(tempGroupList) / float(nRatings)

    if sxx!=0 and syy != 0:
        pearsonCorrelationDict[name]=sxy /sqrt(sxx*syy)
    else:
        pearsonCorrelationDict[name]=0


In [18]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict,orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index=range(len(pearsonDF))
pearsonDF.head()


,similarityIndex,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


In [19]:
topusers=pearsonDF.sort_values(by='similarityIndex',ascending=False)[0:50]
topusers.head()

,similarityIndex,userId
43,1.0,132
34,1.0,18
63,1.0,305
82,1.0,489
86,1.0,525
